<a href="https://colab.research.google.com/gist/m-hamashita/ecbe4991b24aa8f4ce8dcd363971b91e/vscode_ssh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Drive に接続
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# ファイルから ngrok_authtoken と id_rsa.pub を読む
with open('/content/drive/MyDrive/workspace/ngrok_authtoken') as f:
  ngrok_authtoken = f.read().strip()
with open('/content/drive/MyDrive/workspace/id_rsa.pub') as f:
  idrsa_pub = f.read().strip()
!mkdir -p /root/.ssh
!echo $idrsa_pub > /root/.ssh/authorized_keys

!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server x11-apps> /dev/null
!mkdir -p /var/run/sshd
!echo 'X11Forwarding yes' >> /etc/ssh/sshd_config
!echo 'X11UseLocalhost no' >> /etc/ssh/sshd_config
get_ipython().system_raw('/usr/sbin/sshd -D &')

# ngrok 
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip
!rm ngrok-stable-linux-amd64.zip
!./ngrok authtoken $ngrok_authtoken
!pkill -f ngrok
get_ipython().system_raw('./ngrok tcp -region=jp 22 &')
!sleep 2
import urllib.request, json
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
(host, port) = data['tunnels'][0]['public_url'][6:].split(':')
print(f"""Host colab
  HostName {host}
  Port {port}
  User root
  AddKeysToAgent yes
  UseKeychain yes
  ForwardAgent yes
  StrictHostKeyChecking no
  UserKnownHostsFile /dev/null
  ForwardX11 yes
  XAuthLocation /usr/X11/bin/xauth
""")


Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
2048 SHA256:53t147J52Q+xs0c9XmOXnIW0fguSDtjWPQMAreI11AI root@71c373cbd60f (RSA)
Creating SSH2 ECDSA key; this may take some time ...
256 SHA256:TE0k//khsNNVnSMdm4taujDslWuiVNbYVMMvHqf9rsk root@71c373cbd60f (ECDSA)
Creating SSH2 ED25519 key; this may take some time ...
256 SHA256:MCvMKUFTHFHT8/iPT1CCbU4Icx5nlPo5jZrQcEFV4CM root@71c373cbd60f (ED25519)
Created symlink /etc/systemd/system/sshd.service → /lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /lib/systemd/system/ssh.service.
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Host colab
  HostName 0.tcp.jp.ngrok.io
  Port 18241
  User root
  AddKeysToAgent yes
  UseKeychain yes
  ForwardAgent yes
  StrictHostKeyChecking no
  UserKnownHostsFile /dev/null


In [3]:
# シンボリックリンクを作成
!mkdir -p /content/drive/MyDrive/workspace
!ln -sfn /content/drive/MyDrive/workspace /root/workspace

# fish, vim のインストール
!sudo apt-add-repository --yes ppa:fish-shell/release-3
!sudo apt-get update
!sudo apt-get install -y fish vim
# for LC_ALL
!sudo apt-get install language-pack-ja
# ranger 
!pip install ranger-fm
# neovim
!sudo add-apt-repository --yes ppa:neovim-ppa/unstable
!sudo apt-get update
!sudo apt install neovim
# !sudo apt install python-neovim
# !sudo apt install python3-neovim
!pip install pynvim
# dein for neovim
!curl https://raw.githubusercontent.com/Shougo/dein.vim/master/bin/installer.sh > installer.sh
!sh ./installer.sh ~/.cache/dein
!sudo apt-get install nodejs-dev node-gyp libssl1.0-dev
!sudo apt-get install npm
!sudo sudo npm install n -g 
!sudo n lts
# 古い nodejs, npm を削除
!sudo apt purge nodejs npm
!npm install --global yarn
# 上が終わったら neovim 上で :call coc#util#install() を実行
# deno for neovim
!curl -fsSL https://deno.land/install.sh | sh
!
# bat install
!curl -LJO https://github.com/sharkdp/bat/releases/download/v0.20.0/bat-musl_0.20.0_amd64.deb
!sudo dpkg -i bat-musl_0.20.0_amd64.deb 

# config file を drive からサーバーへコピー
!cp /content/drive/MyDrive/workspace/dotfiles/.gitconfig /root/.gitconfig
!cp /content/drive/MyDrive/workspace/dotfiles/.vimrc /root/.vimrc
!cp /content/drive/MyDrive/workspace/id_rsa /root/.ssh/id_rsa
!cp -r /content/drive/MyDrive/workspace/dotfiles/.config/fish /root/.config/
!cp -r /content/drive/MyDrive/workspace/dotfiles/.config/nvim /root/.config/
# なぜか以下をいれると VSCode では開かない (terminal での ssh は OK)
# VSCode は横付けとかしてるからかなぁ
# !chsh -s $(which fish)

# 一旦諦めて .bashrc のなかで fish 起動するように
!echo 'fish' >> /root/.bashrc

# fish の config に追記
!echo 'set -x PATH /usr/local/opt/binutils/bin $PATH' >> /root/.config/fish/config.fish
!echo 'set -x LD_LIBRARY_PATH /usr/lib64-nvidia' >> /root/.config/fish/config.fish
!echo 'set -x PYTHONDONTWRITEBYTECODE 1' >> /root/.config/fish/config.fish
!echo 'set -x TF_CPP_MIN_LOG_LEVEL 2' >> /root/.config/fish/config.fish
!echo 'set -x TERM xterm' >> /root/.config/fish/config.fish
!echo 'set -x DENO_INSTALL "/root/.deno"' >> /root/.config/fish/config.fish
!echo 'set -x PATH "$DENO_INSTALL/bin:$PATH"' >> /root/.config/fish/config.fish


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://ppa

In [4]:
# config に Github の設定を記述
!echo -e "Host github.com\n\tStrictHostKeyChecking no\n\tIdentityFile ~/.ssh/id_rsa" >> ~/.ssh/config
# fzf のインストール
!git clone --depth 1 https://github.com/junegunn/fzf.git ~/.fzf
!sudo ~/.fzf/install --all

Cloning into '/root/.fzf'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 108 (delta 3), reused 43 (delta 1), pack-reused 0
Receiving objects: 100% (108/108), 220.69 KiB | 2.66 MiB/s, done.
Resolving deltas: 100% (3/3), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   666  100   666    0     0   3330      0 --:--:-- --:--:-- --:--:--  3330
100  992k  100  992k    0     0  1687k      0 --:--:-- --:--:-- --:--:-- 1687k
  - Checking fzf executable ... 0.29.0

Generate /root/.fzf.bash ... OK
Update fish_user_paths ... Failed
Symlink /root/.config/fish/functions/fzf_key_bindings.fish ... OK

Update /root/.bashrc:
  - [ -f ~/.fzf.bash ] && source ~/.fzf.bash
    + Added

Update /root/.config/fish/functions/fish_user_key_bindings.fish:
  - fzf_key_bindings
    - Alrea